# Instalação de dependências

In [3]:
! pip install binary gensim==3.6.0

  Using cached gensim-3.6.0-cp38-cp38-linux_x86_64.whl


In [2]:
#!pip3 uninstall gensim -y
! pip3 uninstall numpy -y    
#!apt-get install python3-dev build-essential      
#!pip3 install --upgrade gensim

Found existing installation: numpy 1.19.5
Uninstalling numpy-1.19.5:
  Successfully uninstalled numpy-1.19.5


In [3]:
! pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 1.8 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.23.4 which is incompatible.
tensorflow-gpu 2.9.1 requires absl-py>=1.0.0, but you have absl-py 0.15.0 which is incompatible.
sentence-transformers 2.2.2 requires transformers<5.0.0,>=4.6.0, but you have transformers 3.0.2 which is incompatible.
scipy 1.7.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.4 which is incompatible.
ktrain 0.28.2 requires scikit-learn==0.23.2, but you have scikit-learn 1.1.1 which is incompatible.
ktrain 0.28.2 requires transformers<=4.10.3,>=4.0.0, but you have transformers 3.0.2 which is incompatible.


In [1]:
! pip install gensim==3.6.0
#!pip install gensim==3.7.1
#!pip install gensim==4.0.1

  Using cached gensim-3.6.0-cp38-cp38-linux_x86_64.whl
  Attempting uninstall: gensim
    Found existing installation: gensim 4.1.2
    Uninstalling gensim-4.1.2:
      Successfully uninstalled gensim-4.1.2


# Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import csv
import multiprocessing
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from time import time  # To time our operations

#import logging  # Setting up the loggings to monitor gensim
#logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

/tmp/ipykernel_826500/1128966852.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Treinamento em Pipeline

Construindo vocabulário

In [45]:
# Configurando variáveis de controle e inspeção

cores = multiprocessing.cpu_count()   #Quantidade de cores utilizados no treinamento

t = time()  #Tempo de realização do processo

osm_tables = ['pois_polygons_information', 'pois_roads_information', 'pois_lines_information', 'pois_points_information']
# osm_tables = ['pois_polygons_information']
#osm_tables = ['bins_roads_information', 'bins_lines_information']
#osm_tables = ['bins_polygons_information']

for osm_table in osm_tables:
    n = 400
    w = 0.7
    
    
    
    print('Training model (tuple of weight):', w)

    #ARQUIVO DE DADOS
    file_name = './geographic/train_files/new-york-sl-tuple-geoc2vec-' + osm_table+ '-pfp-c.csv'

    print("loading file:", file_name)
    tuples = pd.read_csv(file_name)

    #Removendo linhas danificadas
    tuples = tuples.dropna()
    tuples = tuples[['center_poi', 'context_osm']]


    #Adaptando para sentenças do word2vec
    sentencesTuples = tuples.values.tolist()

    #Criando estrutura do skip-gram
    p2v_modeltp = Word2Vec(min_count=1,
                            window=1,
                            sg=1, #Skip-gram
                            vector_size=35, #TAMANHO DO VETOR
                            sample=6e-5, 
                            alpha=0.03, 
                            min_alpha=0.0007, 
                            negative=20,
                            workers=cores-1)


    #Criando vocubulário
    p2v_modeltp.build_vocab(sentencesTuples, progress_per=10000)


    #Treinando o modelo
    p2v_modeltp.train(sentencesTuples, total_examples=p2v_modeltp.corpus_count, epochs=1, report_delay=1)

    #Salvando em arquivo
    model_name = './geographic/model/new-york-sl-tuple-geoc2vec-' + osm_table+ 'pfp-c.model'
    
    print('saving file:', model_name)
    p2v_modeltp.save(model_name)


        #except Exception as e:
            #print(str(e))
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))        
print('Process finish.')

Training model (tuple of weight): 0.7
loading file: ./geographic/train_files/new-york-sl-tuple-geoc2vec-pois_polygons_information-pfp-c.csv
saving file: ./geographic/model/new-york-sl-tuple-geoc2vec-pois_polygons_informationpfp-c.model
Training model (tuple of weight): 0.7
loading file: ./geographic/train_files/new-york-sl-tuple-geoc2vec-pois_roads_information-pfp-c.csv
saving file: ./geographic/model/new-york-sl-tuple-geoc2vec-pois_roads_informationpfp-c.model
Training model (tuple of weight): 0.7
loading file: ./geographic/train_files/new-york-sl-tuple-geoc2vec-pois_lines_information-pfp-c.csv
saving file: ./geographic/model/new-york-sl-tuple-geoc2vec-pois_lines_informationpfp-c.model
Training model (tuple of weight): 0.7
loading file: ./geographic/train_files/new-york-sl-tuple-geoc2vec-pois_points_information-pfp-c.csv
saving file: ./geographic/model/new-york-sl-tuple-geoc2vec-pois_points_informationpfp-c.model
Time to train the model: 0.01 mins
Process finish.


In [46]:
p2v_modeltp.wv.most_similar(positive=['Burger Joint'])

[('points_historic_house', 0.5979353189468384),
 ('points_barrier_bollard', 0.5952308773994446),
 ('points_leisure_fitness_centre', 0.5438188910484314),
 ('Ice Cream Shop', 0.5426251292228699),
 ('points_amenity_post_office', 0.5368043184280396),
 ('Garden', 0.5327520966529846),
 ('Clothing Store', 0.5298491716384888),
 ('Drugstore / Pharmacy', 0.5279991030693054),
 ('points_highway_elevator', 0.5232389569282532),
 ('College Academic Building', 0.5219079256057739)]

In [47]:
p2v_modeltp.wv.most_similar(positive=['Park'])

[('Bar', 0.8342798352241516),
 ('points_amenity_bicycle_parking', 0.8123246431350708),
 ('points_covered_no', 0.8061373829841614),
 ('points_barrier_bollard', 0.8020102977752686),
 ('points_highway_crossing', 0.756341278553009),
 ('Coffee Shop', 0.7543727159500122),
 ('points_amenity_charging_station', 0.7541670203208923),
 ('points_barrier_kerb', 0.7423862218856812),
 ('American Restaurant', 0.7271029949188232),
 ('Bowling Alley', 0.7263635396957397)]

In [48]:
p2v_modeltp.wv.most_similar(positive=['Ice Cream Shop'])

[('points_barrier_bollard', 0.7203967571258545),
 ('American Restaurant', 0.7024220824241638),
 ('Residential Building (Apartment / Condo)', 0.6553056836128235),
 ('points_highway_crossing', 0.6469929814338684),
 ('Airport', 0.6395797729492188),
 ('points_covered_no', 0.6297082901000977),
 ('Korean Restaurant', 0.6239877343177795),
 ('points_amenity_bench', 0.6144056916236877),
 ('Home (private)', 0.6076717376708984),
 ('Bank', 0.6037388443946838)]

In [37]:
p2v_modeltp.wv.most_similar(negative=['Ice Cream Shop'])

[('polygons_building_stable', 0.6235930323600769),
 ('polygons_natural_rock', 0.41226842999458313),
 ('polygons_building_food_and_drink', 0.3605819642543793),
 ('Tanning Salon', 0.33694538474082947),
 ('polygons_sport_pilates', 0.31601762771606445),
 ('polygons_surface_acrylic', 0.29497602581977844),
 ('polygons_natural_shingle', 0.29106175899505615),
 ('polygons_surface_mondotrack', 0.2867780327796936),
 ('polygons_leisure_recreation_ground', 0.27139949798583984),
 ('polygons_sport_american_football;soccer;baseball', 0.26619940996170044)]

In [24]:
model_name = './geographic/GEOC2VEC/new-york-sl-tuple-geoc2vec5bin-wgt0.5pfp-concat-c.model'
p2v_modeltp = Word2Vec.load(model_name)